# Module testing

This notebook is used to test and debug functions and classes of *fd-sim* and to illustrate their use.

In [1]:
import os
os.chdir('..')
print('working directory: {}'.format(os.getcwd()))
import numpy as np
import pandas as pd

from responsetimefitting import (
    prepare_data_for_response_time_analysis,
    add_osrm_distance_and_duration,
    model_travel_time,
    fit_dispatch_times,
    fit_turnout_times
)

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)

deployments = pd.read_csv("..\Data\inzetten_2008-heden.csv", sep=";", decimal=",")
incidents = pd.read_csv("..\Data\incidenten_2008-heden.csv", sep=";", decimal=",")
station_locations = pd.read_excel("..\\Data\\kazernepositie en voertuigen.xlsx", sep=";", decimal=".")

working directory: C:\Users\s100385\Documents\JADS Working Files\Final Project\fd-sim


## 1. DeploymentSimulator

### 1.1 responsetimefitting

#### 1.1.1. Prepare data

In [ ]:
df = prepare_data_for_response_time_analysis(incidents, deployments, station_locations)
df.head(1)

#### 1.1.2 Add OSRM estimates of travel distance and duration

In [ ]:
# requires running OSRM
df = add_osrm_distance_and_duration(df, osrm_host="http://192.168.56.101:5000")

In [ ]:
# save file so this process can be skipped next time
df.to_csv("./data/merged_log_with_osrm_info.csv")
df.head()

#### 1.1.3 Model travel time

In [2]:
df = pd.read_csv("./data/merged_log_with_osrm_info.csv")

In [3]:
intercept, coefficient, travel_time_rv = model_travel_time(df)
print("Linear model: {} + {}*t_hat*noise, where t_hat is the OSRM estimate of travel time"
      .format(intercept, coefficient))
print("and the noise is a lognormal random variable with shape={}, loc={}, scale={}"
      .format(travel_time_rv.args[0], travel_time_rv.args[1], travel_time_rv.args[2]))

Linear model: 0.0 + 0.834256928203675*t_hat*noise, where t_hat is the OSRM estimate of travel time
and the noise is a lognormal random variable with shape=6.0945512570972, loc=-506.6351653791085, scale=0.862959531049882


#### 1.1.4 Fit dispatch times per type

In [4]:
dispatch_time_dict = fit_dispatch_times(df)
dispatch_time_dict["Binnenbrand"]

#### 1.1.5 Fit turn-out times per type and station

In [5]:
current_stations = np.append(station_locations["kazerne"].unique(),
                             ["AMSTELVEEN VRIJWILLIG", "GESPREKSGROEPEN", "ONBEKEND"])
turnout_time_dict = fit_turnout_times(df, current_stations)
turnout_time_dict["NICO"]["Buitenbrand"]